<a href="https://colab.research.google.com/github/wolfatthegate/Cuda-Q-tutorials/blob/main/simulating_the_LiH_molecule.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Instructions for Google Colab. You can ignore this cell if you have cuda-q and ipywidgets set up
# and have all the dependent files on your system
# Run this notebook in a CPU runtime
# Uncomment the lines below and execute the cell to install necessary packages and access necessary files.

!pip install cudaq
!pip install ipywidgets
!pip install matplotlib
!pip install qutip
!wget -q https://github.com/nvidia/cuda-q-academic/archive/refs/heads/main.zip
!unzip -q main.zip
!mv cuda-q-academic-main/quick-start-to-quantum/interactive_widget ./interactive_widget

replace cuda-q-academic-main/.gitignore? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C
mv: cannot stat 'cuda-q-academic-main/quick-start-to-quantum/interactive_widget': No such file or directory


1. 🎯 Choose a Fermionic Excitation Term

For LiH in minimal basis (e.g., STO-3G, frozen core), the active space is often mapped to 4 spin-orbitals → 4 qubits.

A common single excitation term: \
$$ a_1^\dagger a_0 - a_0^\dagger a_1 $$ \
is mapped via Jordan-Wigner to: \
$$ i \cdot Y_1 X_0 $$ \
And the corresponding unitary operator is: \
$$ U(\theta) = e^{-i\theta Y_1 X_0} $$  

2. 🔁 Create a Gate for $ e^{-i \theta Y_1 X_0} $

Then implement this composite gate in CUDA-Q.

CUDA-Q does not support arbitrary unitaries directly via matrices, so we decompose using standard gates:

$$ e^{-i \theta Y_1 X_0} = H_0 \cdot RY_1(-\frac{\pi}{2}) \cdot CX_{0,1} \cdot RZ_1(2\theta) \cdot CX_{0,1} \cdot RY_1(\frac{\pi}{2}) \cdot H_0 $$



In [1]:
import cudaq
import numpy as np
from cudaq import spin

def custom_excitation_gate(q, theta):
    """Apply the unitary: exp(-i * θ * Y₁ X₀)"""
    h(q[0])
    ry(q[1], -np.pi / 2)

    cx(q[0], q[1])
    rz(q[1], 2 * theta)
    cx(q[0], q[1])

    ry(q[1], np.pi / 2)
    h(q[0])

@cudaq.kernel
def vqe_ansatz(theta: float):
    q = cudaq.qvector(4)

    # Example entangling + excitation gate
    x(q[0])
    x(q[1])

    # Apply custom fermionic excitation
    custom_excitation_gate(q, theta)

    # Add other gates as needed for full VQE ansatz

A precomputed qubit Hamiltonian for LiH from OpenFermion or [Qiskit Nature]. Following is a sample in qubit form:

In [2]:
from cudaq import spin

# Define Pauli operators
I   = spin.z(0) * 0  # Dummy term to define I
Z0  = spin.z(0)
Z1  = spin.z(1)
Z2  = spin.z(2)
Z3  = spin.z(3)
X0  = spin.x(0)
X1  = spin.x(1)
X2  = spin.x(2)
X3  = spin.x(3)
Y0  = spin.y(0)
Y1  = spin.y(1)
Y2  = spin.y(2)
Y3  = spin.y(3)

# Construct LiH Hamiltonian
H = ( 0.4644 * I
    - 0.1220 * Z0
    - 0.4650 * Z1
    + 0.1200 * Z2
    - 0.3435 * Z3
    + 0.1743 * Z0 * Z1
    + 0.1200 * Z0 * Z2
    + 0.1659 * Z0 * Z3
    + 0.1659 * Z1 * Z2
    + 0.1200 * Z1 * Z3
    + 0.1743 * Z2 * Z3
    + 0.0453 * Y0 * X1 * X2 * Y3
    - 0.0453 * Y0 * Y1 * X2 * X3
    - 0.0453 * X0 * X1 * Y2 * Y3
    + 0.0453 * X0 * Y1 * Y2 * X3
)

In [5]:
import scipy.optimize
from datetime import datetime
import time

start = datetime.now()   # Local start time

def make_ansatz(theta: float):
    @cudaq.kernel
    def ansatz():
        q = cudaq.qvector(4)
        custom_excitation_gate(q, theta)
    return ansatz

# Define the objective function properly
def vqe_objective(params):
    theta = params[0]
    kernel = make_ansatz(theta)
    return cudaq.observe(H, kernel)

# Run the optimization
res = scipy.optimize.minimize(vqe_objective, [0.1], method='COBYLA')

print("Optimal θ:", res.x)
print("Minimum energy:", res.fun)

end_time = datetime.now()

print(f"Started at: {start.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Ended at:   {end.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Total duration: {end - start}")

AttributeError: 'cudaq.mlir._mlir_libs._quakeDialects.cudaq_runtime' object has no attribute 'name'